In [ ]:
# todo: export the model from train.ipynb

# todo: copy import from train.ipynb
# todo: copy normalize() from train.ipynb
# todo: copy predict() from train.ipynb

# todo: inject saved model into predict()
# todo: turn predict() into a GreatAI service

# todo: log prediction into output trace

In [ ]:
from great_ai.utilities import clean
from great_ai import use_model, GreatAI, log_metric
import re

def normalize(text: str) -> str:
    cleaned = clean(text, convert_to_ascii=True).lower()
    return re.sub(r"[^a-z]+", " ", cleaned)

@GreatAI.create
@use_model('financial-sentiment')
def predict_financial_sentiment(text: str, model):
    """Classify news articles into Negative, Neutral, or Positive classes using a linear SVM."""
    text = normalize(text)
    features = model.named_steps["tfidfvectorizer"].transform([text])
    prediction = model.named_steps["sgdclassifier"].predict(features)[0]

    explanation = [
        (feature_name, weight)
        for weight, feature_name in sorted(
            (
                (feature_weight * feature, feature_name)
                for feature_name, feature_weight, feature in zip(
                    model.named_steps["tfidfvectorizer"].get_feature_names_out(),
                    model.named_steps["sgdclassifier"].coef_[list(model.named_steps["sgdclassifier"].classes_).index(prediction)],
                    features.toarray()[0],
                )
                if feature * feature_weight != 0
            ),
            reverse=True,
        )
    ][:10]

    log_metric('prediction', prediction, disable_logging=True)

    return prediction, explanation

In [ ]:
# todo: add integration test (copy metric derivation from train.ipynb)

if __name__ == '__main__':
    from great_ai import query_ground_truth
    import matplotlib.pyplot as plt
    from sklearn import metrics

    test_split = query_ground_truth('test', return_max_count=200)

    traces = predict.process_batch([t.input for t in test_split])
    y_predicted = [t.output[0] for t in traces]
    y_test = [t.output for t in test_split]

    %matplotlib inline
    plt.rcParams["figure.figsize"] = (10, 10)
    plt.rcParams["font.size"] = 16

    print(metrics.classification_report(y_test, y_predicted))
    metrics.ConfusionMatrixDisplay.from_predictions(
        y_true=y_test,
        y_pred=y_predicted,
        xticks_rotation="vertical",
        normalize="pred",
        values_format=".2f",
    )

In [ ]:
# todo: serve prediction model
# todo: open dashboard

!great-ai deploy.ipynb